## INSTALACIÓN TENSORFLOW 2.0

In [ ]:
!pip install packaging
!pip install tf-nightly-2.0-preview

## IMPORTACIONES

In [1]:
from __future__ import absolute_import, division, print_function
import sys
import datetime as dt
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0-dev20190204


In [2]:
tf.executing_eagerly()

True

## EAGER EXECUTION

In [ ]:
#tf.compat.v1.disable_eager_execution()
#tf.executing_eagerly()

In [ ]:
a = tf.constant(3)
# Impresión inmediata del valor de un objeto tf.Tensor
print(a)

### Combinar tf.Tensor con Numpy y uso de tf.print

In [ ]:
def fun(n):
    end = tf.convert_to_tensor(n)
    z = tf.constant(2)
    # Uso de valores de objetos tf.Tensor en estructuras de control de flujo
    for b in range(1, end):
        c = tf.constant(b)
        # La salida de tf.print se dirigirá a la salida stdout de C++ por lo que no se verá en este Jupyter Notebook
        tf.print("El valor de c es: ", c, output_stream=sys.stdout)
        print("El valor de c es: " + str(c))
        # Las operaciones de Numpy aceptan como argumentos objetos tf.Tensor
        d = np.multiply(z, c)
        print("El valor de d es: " + str(d) + " y su tipo es " + str(d.dtype))
        # Transformación de un objeto tf.Tensor en ndarray de Numpy
        e = c.numpy()
        print("El valor de e es: " + str(e))
        
fun(4)

### Autograph @tf.function

Emplear <b>@tf.function</b> para combinar los beneficios de los grafos con Eager Execution:

In [ ]:
# Cualquier función llamada desde otra decorada con @tf.function también será ejecutada en modo de grafo
def sum(x, y):
    return tf.add(x, y)

@tf.function
def my_layer(x, y):
    print(tf.test.is_gpu_available())
    with tf.device("cpu:0"):
        res = sum(x, y)
    return res

x = tf.constant(3)
y = tf.constant(2)
my_layer(x, y)

Emplear <b>@tf.function</b> para transformar elementos de Python en sus correspondientes versiones de Tensorflow. 
* if --> tf.cond
* while / for (Autograph soporta el uso de continue y break)--> tf.while_loop
* for i in data --> data.reduce
* print --> tf.print
* assert --> tf.Assert

In [ ]:
# Recorrer desde 1 hasta i - 1 --> Si elem es divisible entre j imprimir elem
@tf.function
def my_f(i, j):
    for elem in range(1, i):
        if elem % j == 0:
            print(elem)
            
my_f(10, 3)

Imprimir el código generado por <b>Autograph</b>:

In [ ]:
print(tf.autograph.to_code(my_f.python_function, experimental_optional_features=None))

### Cálculo de gradientes con tf.GradientTape

Tensorflow graba en un "tape" todas las operaciones ejecutadas dentro del contexto de un tf.GradientTape.Tensorflow usa ese "tape" para calcular los gradientes de las operaciones grabadas.

In [ ]:
w = tf.Variable([1.0])
with tf.GradientTape() as tape:
    l = w * w
gradient = tape.gradient(l, w)
print(gradient)

### Limitaciones de Autograph
<a src="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md">https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/LIMITATIONS.md</a>

## CARGAR DATOS CON TF.DATA

<b>tf.data.Dataset.from_tensor_slices</b> crea un objeto Dataset cuyos elementos son slices de los tensores de entrada.

In [ ]:
X = np.array([[2, 3], [4, 5], [6, 7]])
y = np.array(["cat", "dog", "fox"])
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset

In [ ]:
for elem_x, elem_y in dataset:
    print(elem_x.numpy(), elem_y.numpy())

El dataset de <b>CIFAR-10</b> está formado por 60000 imágenes de 32x32 píxeles. Hay 10 clases de imágenes y 6000 imágenes
por cada clase repartidas en 50000 imágenes de entrenamiento y 10000 imágenes de test.<br>
Las 10 clases son: <br>
![title](cifar10.png)

In [20]:
def cifar10_dataset():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(10000)
    train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, 
                                                    tf.reshape(tf.one_hot(y, 10), (-1, 10))))
    #train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
    train_dataset = train_dataset.repeat(1)
    
    valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).shuffle(10000)
    valid_dataset = valid_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, 
                                                    tf.reshape(tf.one_hot(y, 10), (-1, 10))))
    valid_dataset = valid_dataset.repeat(1)
    return train_dataset, valid_dataset

train_dataset, valid_dataset = cifar10_dataset()

## USAR CAPAS Y MODELOS DE KERAS

Keras permite emplear diversas aproximaciones para construir la red: Sequential API, Functional API, Subclassing API.
<br>Ahora vamos a ver un ejemplo de <b>Subclassing API</b>:

In [37]:
class KModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(KModel, self).__init__(name='k_model')
        self.num_classes = num_classes
        # Definir las capas de la red
        #self.reshape = tf.keras.layers.Reshape(target_reshape=(28, 28), input_shape=(28,28))
        self.conv1 = tf.keras.layers.Conv2D(64, 5, padding='same', activation=tf.nn.relu, 
                                            #kernel_initializer=tf.compat.v1.variance_scaling_initializer, 
                                            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        # variance_scaling: samples are drawn from a truncated normal distribution with a mean of zero and a 
        # standard deviation stddev = sqrt(scale / n) where n is: 
        # number of input units in the weight tensor
        # regularization_l2 += tf.python.ops.math_ops.reduce_sum(self.l2 * math_ops.square(x))
        self.conv2 = tf.keras.layers.Conv2D(64, 5, padding='same', activation=tf.nn.relu, 
                                            #kernel_initializer=tf.compat.v1.variance_scaling_initializer, 
                                            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(750, activation=tf.nn.relu, 
                                         #kernel_initializer=tf.compat.v1.variance_scaling_initializer,
                                         kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.fc2 = tf.keras.layers.Dense(num_classes)
        self.softmax = tf.keras.layers.Softmax()
        
    def call(self, x):
        x = self.maxpool(self.conv1(x))
        x = self.batch_norm(x)
        x = self.maxpool(self.conv2(x))
        x = self.batch_norm(x)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)      

### Entrenamiento del modelo

In [38]:
model = KModel(num_classes=10)
# Keras accuracy K.mean(K.equal(y_true, K.round(y_pred)))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))

]

model.fit(train_dataset, epochs=1, validation_data=valid_dataset, validation_steps=3, 
          callbacks=callbacks)

1563/1563==============================] - 310s 198ms/step


### Evaluación del modelo

In [39]:
#model.evaluate(data, labels, batch_size=32)
model.evaluate(valid_dataset, steps=30)

30/30==============================] - 1s 41ms/step - loss: 7.3224 - acc: 0.1156


[7.322403685251872, 0.115625]

### Predicción

In [40]:
i = True
for x , y in valid_dataset:
    if i == True:
        elem_x = x
        elem_y = y
        i = False

In [41]:
res = model.predict(elem_x, batch_size=32)
max_ind_res = tf.argmax(res, axis=1)
print(max_ind_res)

tf.Tensor([0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(32,), dtype=int64)


In [42]:
print(tf.argmax(elem_y, axis=1))

tf.Tensor([4 6 8 0 5 4 1 2 0 9 6 4 2 4 6 5 9 7 7 4 6 5 0 1 9 0 3 1 9 0 9 7], shape=(32,), dtype=int64)


In [43]:
print(res)

[[9.20445085e-01 4.74516681e-04 4.39676866e-02 3.49011533e-02
  2.35347656e-07 1.66804617e-04 2.64197342e-06 4.20839683e-07
  4.14171409e-05 5.65345246e-08]
 [7.94559717e-01 3.11323232e-03 1.79192171e-01 2.25958172e-02
  1.34762843e-06 2.39591172e-04 5.11891994e-05 7.39940651e-06
  2.37632601e-04 1.91255140e-06]
 [7.75377095e-01 7.15321079e-02 1.35324135e-01 1.69503745e-02
  4.46334155e-07 1.39353069e-04 2.80247186e-05 2.04888984e-05
  5.89387957e-04 3.84870909e-05]
 [9.94001448e-01 7.53943168e-04 4.97061620e-03 6.93867187e-05
  2.38935627e-08 6.76246771e-07 1.89199852e-07 2.56242743e-08
  2.02818424e-04 8.25951304e-07]
 [5.81852794e-01 1.76884903e-04 1.20578505e-01 2.93874979e-01
  4.21841122e-07 3.48766893e-03 1.37631569e-05 2.29540433e-06
  1.27146195e-05 7.60194041e-08]
 [7.41658807e-01 4.71866661e-04 2.26233169e-01 3.08853909e-02
  1.68717133e-06 5.86691138e-04 3.00097054e-05 2.62865015e-05
  1.04963648e-04 1.10801352e-06]
 [8.88136387e-01 1.75037177e-03 6.64514005e-02 4.31174710e

In [44]:
print(elem_y)

tf.Tensor(
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

### Salvar los pesos

In [48]:
# Salvar los pesos en un fichero Checkpoint de Tensorflow 
model.save_weights('./kmodel_tf')
# Restaurar el estado del modelo. Esto requiere que el modelo tenga la misma arquitectura que la red de los pesos
model.load_weights('./kmodel_tf')

# Salvar los pesos en formato HDF5 de Keras
model.save_weights('./kmodel_k.h5', save_format='h5')
# Restaurar el estado del modelo. Esto requiere que el modelo tenga la misma arquitectura que la red de los pesos
model.load_weights('./kmodel_k.h5')

### Salvar el modelo completo

In [49]:
# SavedModel es un formato serializado que contendrá las variables, el grafo y los metadatos del grafo.
# Permite que herramientas de alto nivel como tensorflow/serving, la herramienta de línea de comandos saved_model
model_path = "/home/turtlebot/full_model"
tf.saved_model.save(model, model_path)